In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
pathdata17zip = '/content/drive/My Drive/Code_Attribution/data17.zip'

In [0]:
import zipfile
zip_ref = zipfile.ZipFile(pathdata17zip, 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

In [0]:
datasetPath = '/tmp/data17'

In [0]:
from keras.layers import Input, Dense, Conv1D, MaxPooling1D, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau, TensorBoard
from keras.models import Model
import keras.backend as K

In [0]:
import os

path = datasetPath

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.cpp' in file:
            files.append(os.path.join(r, file))

# for f in files:
#     print(f)

In [0]:
len(files)

56837

In [0]:
outputvar = {}
counter = 0
for x in files:
    user = x.split('/')[-2]
    if user not in outputvar.keys():
        outputvar[user] = counter
        counter += 1

In [0]:
lengths = []
minLength = 2000

In [0]:
def padding(fname):
    f = open(fname, "r")
    y = f.read()
    f.close()
    
    origy = y
    while (len(y) < minLength):
        y += origy
        
    f = open(fname, "w")
    f.write(y)
    f.close()
    return y

In [0]:
for i in range(0,3):
    print(i)
    lengths = []
    count = 0
    for fil in files:
        f = open(fil, "r")
        x = f.read()
        f.close()
        if(len(x) <= 100):
            files.remove(fil)
        elif len(x) == 0:
            print(fil)
            files.remove(fil)
        elif(len(x) < minLength):
            x = padding(fil)
            lengths.append(len(x))
        else:
            lengths.append(len(x))


0
1
2


In [0]:
print(max(lengths))
print(min(lengths))
print(len(lengths))
print(len(files))

95319
2000
56808
56808


In [0]:
import numpy as np

In [0]:
inp = np.array(files)
names = []
for fil in files:
    names.append(outputvar[fil.split('/')[-2]])
out = np.array(names)
print(inp[:10])
print(out[:10])

['/tmp/data17/Izazz/untitled0.cpp' '/tmp/data17/deone/game20.cpp'
 '/tmp/data17/deone/Game0.cpp' '/tmp/data17/deone/Game1.cpp'
 '/tmp/data17/cthbst/pB0.cpp' '/tmp/data17/cthbst/pC2.cpp'
 '/tmp/data17/cthbst/pC1.cpp' '/tmp/data17/cthbst/pA1.cpp'
 '/tmp/data17/cthbst/pB1.cpp' '/tmp/data17/cthbst/pC0.cpp']
[0 1 1 1 2 2 2 2 2 2]


In [0]:
from sklearn.model_selection import train_test_split

In [0]:
x_train, X_test, y_train, Y_test = train_test_split(inp, out, test_size=0.2, random_state = 42)
X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train, test_size=0.2, random_state = 42)

In [0]:
def walk(path, vocab, count):
    for r, d, f in os.walk(path):
        for file in f:
            f = open(os.path.join(r, file))
            x = f.read()
            for line in x:
                for letter in line:
                    if letter not in vocab.keys():
                        vocab[letter] = count[0]
                        count[0] += 1


In [0]:
vocab = {}
count = [0]
path = datasetPath
for r, d, f, in (os.walk(path)):
    for i, dir in enumerate(d):
        walk(os.path.join(r, dir), vocab, count)
print(count)

[1468]


In [0]:
def getFiles(path):
    matrix = []
    for p in path:
        tmp = np.zeros((50, 40), dtype = int)
        f = open(p)
        for i in range(0,50):
            for j in range(0, 40):
                tmp[i][j] = vocab[f.read(1)]
        matrix.append(tmp)
    return np.array(matrix)

In [0]:
num_classes = len(outputvar)
input_shape = (50, 40)
batch_size = 1024
def data_generator(file_paths, author_numbers):
    batch_start = 0
    batch_end = batch_start + batch_size
    n = file_paths.shape[0]
    indexes = np.arange(0, n, batch_size)
    if n % batch_size != 0:
        indexes = indexes[:-1] 
    while True:
        np.random.shuffle(indexes)
        for ind in indexes:
            batch_start = ind
            batch_end = batch_start + batch_size
            myFiles = file_paths[batch_start:batch_end]
            batch_files = getFiles(myFiles)
            batch_authors = author_numbers[batch_start:batch_end]
            batch_x = np.zeros((batch_size, 50, 40))
            tmp = np.zeros((batch_size, num_classes))
            batch_x = np.array( batch_files )
            for i, j in enumerate(batch_authors):
                tmp[i][j] = 1
            batch_y = np.array( tmp )
            yield( batch_x, batch_y )

In [0]:
input_fil = Input(shape=(input_shape), name='input_fil')
conv = Conv1D(64, kernel_size=7, strides = 2,  activation='relu')(input_fil)
drop = Dropout(0.1)(conv)
maxP = MaxPooling1D()(drop)
flat = Flatten()(maxP)
output_class = Dense(num_classes, activation='softmax')(flat)

model = Model(inputs=input_fil, outputs=output_class)

W0727 08:04:17.069743 140452118001536 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0727 08:04:17.078921 140452118001536 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0727 08:04:17.095486 140452118001536 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



In [0]:
adam = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', 
              optimizer=adam, 
              metrics=['accuracy'])

model.summary()

W0727 08:04:20.096041 140452118001536 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0727 08:04:20.104935 140452118001536 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_fil (InputLayer)       (None, 50, 40)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 22, 64)            17984     
_________________________________________________________________
dropout_1 (Dropout)          (None, 22, 64)            0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 11, 64)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 704)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 11609)             8184345   
Total params: 8,202,329
Trainable params: 8,202,329
Non-trainable params: 0
_________________________________________________________________


In [0]:
train_gen = data_generator(X_train, Y_train)
test_gen = data_generator(X_test, Y_test)
val_gen = data_generator(X_val, Y_val)

In [0]:
print(X_train[:10])
print(X_test[:10])
print(Y_train[:10])
print(Y_test[:10])

['/tmp/data17/Anton.Lunyov/main1.cpp' '/tmp/data17/caojiangxia/main0.cpp'
 '/tmp/data17/y4st/A1.cpp' '/tmp/data17/Balajiganapathi/c0.cpp'
 '/tmp/data17/go.goo.gle/CDJ1A171.cpp' '/tmp/data17/kyoae/P21.cpp'
 '/tmp/data17/luki4824/Ratatouille0.cpp' '/tmp/data17/Triracle/A0.cpp'
 '/tmp/data17/asv/BathroomStalls10.cpp' '/tmp/data17/Fcdkbear/Source1.cpp']
['/tmp/data17/CookiC/Problem B0. Parenting Partnering0.cpp'
 '/tmp/data17/ssor96/b0.cpp' '/tmp/data17/ardo/main0.cpp'
 '/tmp/data17/andzaytsev/C0.cpp' '/tmp/data17/bbyalcinkaya/cruise1.cpp'
 '/tmp/data17/Vishal4556/AmpSyr0.cpp' '/tmp/data17/calee0219/pB1.cpp'
 '/tmp/data17/pk.tautolo/B0.cpp' '/tmp/data17/andyshih12/B1.cpp'
 '/tmp/data17/shivammittal99/B0.cpp']
[ 8130 11102  5026  3028  4893  2086  9479 10826  4238  3643]
[ 5848   931  3164  9426  1615 11213   350   764  4528  2367]


In [0]:
hist1 = model.fit_generator(train_gen, epochs=10, steps_per_epoch=len(Y_train)//batch_size, 
                           validation_data=val_gen, validation_steps=len(Y_val)//batch_size, verbose=1)

W0727 08:04:33.753188 140452118001536 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
35/35 [==============================] - 52s 1s/step - loss: 10.3319 - acc: 0.0016 - val_loss: 9.2861 - val_acc: 0.0054
Epoch 2/10
35/35 [==============================] - 45s 1s/step - loss: 8.4675 - acc: 0.0398 - val_loss: 8.7507 - val_acc: 0.0562
Epoch 3/10
35/35 [==============================] - 45s 1s/step - loss: 5.3861 - acc: 0.2641 - val_loss: 7.9367 - val_acc: 0.1962
Epoch 4/10
35/35 [==============================] - 45s 1s/step - loss: 2.8402 - acc: 0.5418 - val_loss: 7.3712 - val_acc: 0.3274
Epoch 5/10
35/35 [==============================] - 44s 1s/step - loss: 1.5315 - acc: 0.7341 - val_loss: 7.5068 - val_acc: 0.3739
Epoch 6/10
35/35 [==============================] - 45s 1s/step - loss: 0.9472 - acc: 0.8328 - val_loss: 7.6913 - val_acc: 0.3944
Epoch 7/10
35/35 [==============================] - 45s 1s/step - loss: 0.6408 - acc: 0.8878 - val_loss: 7.6113 - val_acc: 0.4204
Epoch 8/10
35/35 [==============================] - 44s 1s/step - loss: 0.4405 - acc: 0.9

In [0]:
xtestdata = getFiles(X_test)
print(xtestdata.shape)

(11362, 50, 40)


In [0]:
#X_test.shape

In [0]:
#pre = model.predict_generator(test_gen, steps = np.ceil(X_test.shape[0]/batch_size))

In [0]:
#pre.shape

In [0]:
preds = model.predict(xtestdata)
print(preds.shape)

(11362, 11609)


In [0]:
myPreds = []
for i in preds:
    myPreds.append(np.argmax(i))
myPreds = np.array(myPreds)
print(myPreds.shape)

(11362,)


In [0]:
count = 0
for i in range(Y_test.shape[0]):
    if myPreds[i] == Y_test[i]:
        count += 1

acc = (count / Y_test.shape[0]) * 100
print("Accuracy", acc)

Accuracy 43.46945960218272
